In [3]:
import scipy as sp
import statistics as stat
import matplotlib.pyplot as plt
import seaborn
import pandas_datareader as pdr
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [4]:
class StockAnalyzer:
    def __init__(self, stocks=[], weights=[], source="yahoo", date="", betayears=5, market="SPY", monthly=True):
        self.stocks = stocks
        self.weights = weights
        self.source = source
        self.betayears = betayears
        self.market = market
            
        if date=="":
            self.date = datetime.date.today()
        else:
            self.date = datetime.strpTime(date)
        
        self.start = self.date + relativedelta(years=-self.betayears)
        
        #if monthly:
            
        
        self.prices = pdr.DataReader(self.stocks, data_source = self.source, start = self.start, end = self.date)
        self.prices = self.prices["Adj Close"]
        self.market = pdr.DataReader(self.market, data_source = self.source, start = self.start, end = self.date)
        self.market = self.market["Adj Close"]
        
    def portfoliobeta(self, excel=False):
        benchmark = self.market.pct_change()[1:]
        returns = self.prices.pct_change()[1:]
        portfolio = (returns * self.weights).sum(axis=1)
        (beta, alpha) = sp.stats.linregress(benchmark.values, portfolio.values)[0:2]
        stockbeta = pd.DataFrame()
        for stock in returns.columns:
            (beta, alpha) = sp.stats.linregress(benchmark.values, returns[stock].values)[0:2]
            stockbeta = stockbeta.append({"Stock":stock, "beta":beta, "alpha":alpha}, ignore_index=True)
        print(stockbeta)
        print(f"Portfolio beta: {beta}")
        
        if excel:
            stockbeta.to_excel('./stockbetas.xlsx', sheet_name = "Betas", index=False)

In [5]:
analyze = StockAnalyzer(["CCL","AAPL","NCLH","BABA","SCHX","DTE","QFIN"], [0.1681,0.1485,0.2986,0.1831,0.1494,0.0523, 0.1909])
analyze.portfoliobeta()

TypeError: string indices must be integers